## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [13]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad

    def __eq__(self, other):
        # Compara productos por su código
        return isinstance(other, Producto) and self.codigo == other.codigo

    def __hash__(self):
        # Usa el código del producto para generar el hash
        return hash(self.codigo)

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if 10 <= valor <= 10000:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if 0 <= valor <= 1000:
            self._cantidad = valor

    def valorTotal(self):
        return self._precio * self._cantidad

In [14]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [15]:
class Oferta:
    def __init__(self, descripcion='', codigos=None, tipos=None):
        self.descripcion = descripcion
        self.codigos = codigos if codigos is not None else []
        self.tipos = tipos if tipos is not None else []

    def esAplicable(self, producto):
        # Verifica si la oferta es aplicable al producto por código o tipo
        aplicable_por_codigo = producto.codigo in self.codigos
        aplicable_por_tipo = producto.tipo in self.tipos
        return aplicable_por_codigo or aplicable_por_tipo


class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion='', codigos=None, tipos=None):
        super().__init__(descripcion, codigos, tipos)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        # Verifica si la oferta es aplicable
        if self.esAplicable(producto):
            # Calcula el monto de descuento, no el precio final
            return producto.precio * cantidad * (self.descuento / 100)
        return 0  # Si no es aplicable, devuelve 0 descuento


class Oferta2x1(Oferta):
    def __init__(self, descripcion='', codigos=None, tipos=None):
        super().__init__(descripcion, codigos, tipos)

    def calcularDescuento(self, producto, cantidad):
        # Verifica si la oferta es aplicable
        if self.esAplicable(producto):
            # Si aplica el 2x1, calcula cuántos productos se pagan
            cantidad_a_pagar = (cantidad // 2) + (cantidad % 2)
            return producto.precio * (cantidad - cantidad_a_pagar)  # Descuento es lo que no se paga
        return 0  # Si no es aplicable, devuelve 0 descuento


In [16]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [17]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = {}  # Diccionario de productos por código
        self.ofertas = []    # Lista de ofertas disponibles

    def agregar(self, *productos):
        for producto in productos:
            self.productos[producto.codigo] = producto

    def vender(self, producto, cantidad):
        if producto.codigo in self.productos:
            self.productos[producto.codigo].cantidad -= cantidad

    def guardar(self, archivo):
        with open(archivo, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos.values():
                writer.writerow([
                    producto.codigo,
                    producto.nombre,
                    producto.precio,
                    producto.tipo,
                    producto.cantidad
                ])

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Saltamos la cabecera
            for row in reader:
                if len(row) == 5:  # Nos aseguramos que la fila tenga todos los campos
                    codigo, nombre, precio, tipo, cantidad = row
                    try:
                        producto = Producto(
                            codigo,
                            nombre,
                            float(precio),
                            tipo,
                            int(cantidad)
                        )
                        catalogo.productos[codigo] = producto
                    except ValueError as e:
                        print(f"Error al procesar la fila {row}: {e}")
        return catalogo

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscar(self, codigo):
        return self.productos.get(codigo)

    def buscarOferta(self, producto, cantidad):
        mejor_oferta = None
        mayor_descuento = 0
        
        for oferta in self.ofertas:
            if oferta.esAplicable(producto):
                descuento = oferta.calcularDescuento(producto, cantidad)
                if descuento > mayor_descuento:
                    mayor_descuento = descuento
                    mejor_oferta = oferta
        
        return mejor_oferta

    def calcularDescuento(self, producto, cantidad):
        mejor_descuento = 0
        
        for oferta in self.ofertas:
            if oferta.esAplicable(producto):
                descuento = oferta.calcularDescuento(producto, cantidad)
                mejor_descuento = max(mejor_descuento, descuento)
        
        return mejor_descuento

    def informe(self):
        tipos = sorted(set(p.tipo for p in self.productos.values()))
        precio_promedio_total = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0

        lineas = []
        # Formato exacto solicitado
        lineas.append("INFORME CATALOGO")
        lineas.append(f"Cantidad de productos:   {self.cantidadProductos}")
        lineas.append(f"Cantidad de unidades:    {self.cantidadUnidades}")
        lineas.append(f"Precio Promedio:       $ {precio_promedio_total:.2f}")
        lineas.append(f"Valor total:           $ {self.valorTotal:.2f}")
        lineas.append("Tipos de productos: ")

        for tipo in tipos:
            productos_tipo = [p for p in self.productos.values() if p.tipo == tipo]
            unidades_tipo = sum(p.cantidad for p in productos_tipo)
            valor_total_tipo = sum(p.valorTotal() for p in productos_tipo)
            precio_promedio_tipo = valor_total_tipo / unidades_tipo if unidades_tipo > 0 else 0
            # Formato alineado de tipos de productos
            lineas.append(f"  - {tipo:<18} :  {unidades_tipo}u x $ {precio_promedio_tipo:.2f}")

        lineas.append("Ofertas:")
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                lineas.append("  - Oferta 2x1")
            elif isinstance(oferta, OfertaDescuento):
                lineas.append(f"  - Descuento {oferta.descuento}%")
        
        # Imprimir el informe
        informe_str = "\n".join(lineas)
        print(informe_str)
        
        # Devolver el informe como string
        return informe_str



    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos.values())

    @property
    def valorTotal(self):
        return sum(producto.valorTotal() for producto in self.productos.values())

In [18]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

INFORME CATALOGO
Cantidad de productos:   4
Cantidad de unidades:    97
Precio Promedio:       $ 1684.54
Valor total:           $ 163400.00
Tipos de productos: 
  - galleta            :  67u x $ 1856.72
  - gaseosa            :  30u x $ 1300.00
Ofertas:
  - Oferta 2x1
  - Descuento 10%
Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [19]:
import re

class Cliente:
    def __init__(self, nombre, cuit):
        # Validar el nombre y el CUIT al inicializar
        self._nombre = None
        self._cuit = None
        
        self.nombre = nombre
        self.cuit = cuit

    # Propiedad para obtener el nombre
    @property
    def nombre(self):
        return self._nombre

    # Propiedad para establecer el nombre con validación
    @nombre.setter
    def nombre(self, valor):
        if 1 <= len(valor) <= 100:  # Validar que el nombre tenga entre 1 y 100 caracteres
            self._nombre = valor

    # Propiedad para obtener el CUIT
    @property
    def cuit(self):
        return self._cuit

    # Propiedad para establecer el CUIT con validación
    @cuit.setter
    def cuit(self, valor):
        # Usar una expresión regular para validar el formato del CUIT (XX-XXXXXXXX-X)
        if re.fullmatch(r'\d{2}-\d{8}-\d', valor):
            self._cuit = valor



In [20]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [21]:
from datetime import datetime

class Factura:
    _ultimo_numero = 0
    
    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero
    
    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero
    
    def __init__(self, catalogo, cliente):
        self.numero = self.nuevoNumero()
        self.fecha = datetime.now()
        self.catalogo = catalogo
        self.cliente = cliente
        self.items = {}  # Diccionario para almacenar producto: cantidad
        
    def agregar(self, producto, cantidad):
        if producto is None or producto.cantidad < cantidad:
            return
            
        if producto in self.items:
            self.items[producto] += cantidad
        else:
            self.items[producto] = cantidad
            
        # Actualizar el stock en el catálogo
        self.catalogo.vender(producto, cantidad)
            
    @property
    def cantidadProductos(self):
        return len(self.items)
        
    @property
    def cantidadUnidades(self):
        return sum(self.items.values())
        
    @property
    def subtotal(self):
        total = 0
        for producto, cantidad in self.items.items():
            total += producto.precio * cantidad
        return total
        
    @property
    def descuentos(self):
        total_descuentos = 0
        for producto, cantidad in self.items.items():
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            total_descuentos += descuento
        return total_descuentos
        
    @property
    def total(self):
        return self.subtotal - self.descuentos
        
    def imprimir(self):
        resultado = []
        resultado.append(f"Factura: {self.numero}")
        resultado.append(f"Fecha  : {self.fecha.strftime('%d/%m/%Y')}")
        resultado.append(f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n")
        
        for producto, cantidad in self.items.items():
            subtotal = producto.precio * cantidad
            resultado.append(f"- {cantidad} {producto.nombre:<25} x ${producto.precio:.2f} = ${subtotal:.2f}")
            
            oferta = self.catalogo.buscarOferta(producto, cantidad)
            if oferta:
                if isinstance(oferta, OfertaDescuento):
                    descuento = producto.precio * cantidad * oferta.descuento / 100
                    resultado.append(f"      Descuento {oferta.descuento}%{'':<35} - ${descuento:.2f}")
                elif isinstance(oferta, Oferta2x1):
                    resultado.append(f"      Oferta 2x1{'':<35}")

        resultado.append(f"\n                                             Subtotal:   ${self.subtotal:.2f}")
        resultado.append(f"                                             Descuentos: ${self.descuentos:.2f}")
        resultado.append(f"                                             -----------------------")
        resultado.append(f"                                             TOTAL:      ${self.total:.2f}")
        
        # Aseguramos los saltos de línea
        informe_str = "\n".join(resultado)
        print(informe_str)  # Imprime el informe en consola
        return informe_str  # Retorna el informe como string


In [22]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Factura: 103
Fecha  : 05/10/2024
Cliente: Juan Perez (20-12345678-9)

- 10 Coca Cola                 x $1500.00 = $15000.00
      Descuento 10%                                    - $1500.00
- 3 Sonrisa                   x $1200.00 = $3600.00
      Oferta 2x1                                   

                                             Subtotal:   $18600.00
                                             Descuentos: $2700.00
                                             -----------------------
                                             TOTAL:      $15900.00
Prueba pasada exitosamente!


In [23]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
